<a href="https://colab.research.google.com/github/shiva1999/mlops_task4/blob/master/Face_Recoginition_with_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Recognition using VGG16

### Loading the VGG16 Model

In [ ]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

### Inpsecting each layer

In [ ]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

### Freezing of layers

In [ ]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

### Let's make a function that returns our FC Head

In [ ]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(4096, activation = "relu")(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(512, activation = "relu")(top_model)
    top_model = Dense(256, activation = "relu")(top_model)
    top_model = Dense(32, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [ ]:
model.input

In [ ]:
model.layers

### Let's add our FC Head back onto VGG

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

### Loading our Face Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'drive/My Drive/wiki_crop/'
validation_data_dir = 'drive/My Drive/a_test/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
batchsize = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batchsize,
        class_mode='categorical',
        shuffle=False)

### Training our top layers

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          verbose = 1,
                          patience= 3,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.0001),
              metrics = ['accuracy'])

nb_train_samples = 2100 
nb_validation_samples = 90
epochs = 5
train_batch_size = 32
val_batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // train_batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // val_batch_size)

modelnew.save("face_vgg.h5")

Epoch 1/5
65/65 [==============================] - 31s 469ms/step - loss: 0.4662 - accuracy: 0.7758 - val_loss: 1.1199 - val_accuracy: 0.5250

Epoch 00001: val_loss improved from inf to 1.11989, saving model to face_vgg.h5
Epoch 2/5
65/65 [==============================] - 30s 458ms/step - loss: 0.4575 - accuracy: 0.7725 - val_loss: 0.7468 - val_accuracy: 0.4187

Epoch 00002: val_loss improved from 1.11989 to 0.74682, saving model to face_vgg.h5
Epoch 3/5
65/65 [==============================] - 29s 448ms/step - loss: 0.4439 - accuracy: 0.7879 - val_loss: 1.8926 - val_accuracy: 0.3562

Epoch 00003: val_loss did not improve from 0.74682
Epoch 4/5
65/65 [==============================] - 30s 463ms/step - loss: 0.4625 - accuracy: 0.7783 - val_loss: 1.1014 - val_accuracy: 0.4938

Epoch 00004: val_loss did not improve from 0.74682
Epoch 5/5
65/65 [==============================] - 29s 453ms/step - loss: 0.4462 - accuracy: 0.7884 - val_loss: 0.9454 - val_accuracy: 0.4187
Restoring model weig